# Create Model & Load Saved Weights

In [1]:
# load classes from python file resnet_18.py
from resnet_18 import ResNet18

def getModel():
    model = ResNet18(10)
    model.build(input_shape = (None,32,32,3))
    model.compile(optimizer = "adam", loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model

In [2]:
# build model and load weights
model1 = getModel()

# load weights
model1.load_weights('resnet_18.h5')

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


# Prediction

In [3]:
from PIL import Image
import numpy as np

def predict(classifier, image_path):
  # class label
  labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

  image = Image.open(image_path)
  #plt.show(image)

  # resize image
  im = image.resize((32,32))
  
  # normalize the data.
  im = np.array(im).astype('float32') / 255.0
  im = np.expand_dims(im, axis=0)

  # close image
  image.close()

  # classify image
  pred = classifier.predict(im)
  index = np.argmax(pred, axis=1)[0]

  print(image_path.split('\\')[-1], ':', labels[index], pred[0][index])

In [4]:
import glob

# classify all images in folder 'test_images'
for img in glob.glob("../test_images/*.jpg"):
    predict(model1, img)

car.jpg : automobile 1.0
cat.jpg : dog 0.9996878
horse.jpg : dog 0.46701697
